### <strong>主題:
啤酒評論評分預測 - 回歸模型建構
### <strong>說明:
完成了先前的課程之後，我們了解到有些問題其實是比較適合回歸模型的，比方說評分預測的問題 <br />
，因此我們這次以回歸模型，重新評估其各項屬性(apperance, aroma, overall, palate, taste)得分<br />
。另外，啤酒問題為多標籤問題(multi-label regression)，也必須特別注意。
### <strong>題目
1. 參照前次分類模型的結構，將分類模型修改為回歸模型<br />
(完成底下的BeerDataset與create_data_loader)
2. 修改完後，進行模型訓練，確認所修改的架構不會產生錯誤。

#### <strong>提示1: 若同學因GPU限制無法快速訓練，可以考慮調低訓練回合數，MAX_LEN，或選擇較小的bert模型。

In [1]:
# 連接個人資料 讀取 ＰＴＴ 訓練資料和儲存模型
#先連接自己的GOOGLE DRIVE 為了要儲存資料和訓練模型
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

# Current directory
print(os.getcwd())

# change directory
os.chdir('/content/drive/MyDrive/python_training/NLP100Days-part2/project_2_5/')
print(os.getcwd())

/content
/content/drive/MyDrive/python_training/NLP100Days-part2/project_2_5


In [3]:
!pip install torch
!pip install transformers
#!pip install -q transformers
#!pip install transformers==3
# 設定 torchtext 版本 安裝完必須重新啟動執行階段
!pip install torchtext==0.6.0
#!pip install -r requirements.txt

     |████████████████████████████████| 2.6 MB 4.7 MB/s 
     |████████████████████████████████| 895 kB 44.4 MB/s 
     |████████████████████████████████| 3.3 MB 54.0 MB/s 
     |████████████████████████████████| 636 kB 62.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 64 kB 2.0 MB/s 
     |████████████████████████████████| 1.2 MB 10.7 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0


In [4]:
import torch
import transformers
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
from transformers import BertModel, BertTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup

import warnings
warnings.filterwarnings('ignore')

In [5]:
PRE_TRAINED_MODEL_NAME = "bert-base-cased"
BATCH_SIZE = 16
MAX_LEN = 255
EPOCHS = 5 #10

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
TOKENIZER = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [6]:
print(DEVICE)

cuda:0


In [7]:
class BeerDataset(Dataset):
    """
    將資料集轉換為後續data DataLoader 需求的 pytorch Dataset形式
    Convert beer review dataframe into torch dataset instance
    """
    def __init__(self,
                 comments,
                 appearance_target,
                 aroma_target,
                 overall_target,
                 palate_target,
                 taste_target, max_len):
        self.comments = comments
        self.appearance_target = appearance_target
        self.aroma_target = aroma_target
        self.overall_target = overall_target
        self.palate_target = palate_target
        self.taste_target = taste_target
        self.max_len = max_len

    def __len__(self):
        return len(self.comments)

    def __getitem__(self, item):
        comment = str(self.comments[item])
        # 須完成之部分
        appearance_target = self.appearance_target[item]
        aroma_target = self.aroma_target[item]
        overall_target = self.overall_target[item]
        palate_target = self.palate_target[item]
        taste_target = self.taste_target[item]
        encoding = TOKENIZER.encode_plus(
            comment,
            max_length=self.max_len,
            truncation=True,
            add_special_tokens=True,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'comment': comment,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),# 須完成之部分
            'appearance_target': torch.Tensor([appearance_target]),# 須完成之部分
            'aroma_target': torch.Tensor([aroma_target]),# 須完成之部分
            'overall_target': torch.Tensor([overall_target]),# 須完成之部分
            'palate_target': torch.Tensor([palate_target]),# 須完成之部分
            'taste_target': torch.Tensor([taste_target])# 須完成之部分
        }

In [8]:
def create_data_loader(dataframe, max_len, batch_size):
    """
    將pytorch Dataset形式資料集包裝為data DataLoader
    convert dataset to pytorch dataloader format object
    """
    dataset = BeerDataset(
        # 須完成之部分
        comments=dataframe['review/text'],
        appearance_target=dataframe.review_appearance,
        aroma_target=dataframe.review_aroma,
        overall_target=dataframe.review_overall,
        palate_target=dataframe.review_palate,
        taste_target=dataframe.review_taste,
        max_len=max_len
    )
    return DataLoader(
        dataset,
        batch_size=batch_size
    )


In [9]:
class BeerRateRegressor(nn.Module):
    """
    啤酒評論評分回歸模型的主體
    Beer sentiment main model for review sentiment analyzer
    """
    def __init__(self):
        super(BeerRateRegressor, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.drop = nn.Dropout(p=0.2)
        aspects = ['appearance', 'aroma', 'overall', 'palate', 'taste']
        self.aspect_outs = nn.ModuleDict({
            aspect: nn.Linear(self.bert.config.hidden_size, 1)
            for aspect in aspects
        })

        # self.appearance_out = nn.Linear(self.bert.config.hidden_size, 1)
        # self.aroma_out = nn.Linear(self.bert.config.hidden_size, 1)
        # self.overall_out = nn.Linear(self.bert.config.hidden_size, 1)
        # self.palate_out = nn.Linear(self.bert.config.hidden_size, 1)
        # self.taste_out = nn.Linear(self.bert.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask):
        bertOutput = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        # 須完成之部分
        out = self.drop(bertOutput['pooler_output'])
        aspect_outputs = {
            aspect: aspect_out(out)
            for aspect, aspect_out in self.aspect_outs.items()
        }

        return aspect_outputs

        # return {
        #     "appearance": appearance_output,
        #     "aroma": aroma_output,
        #     "overall": overall_output,
        #     "palate": palate_output,
        #     "taste": taste_output,
        # }

In [10]:
def train_epoch(model,
                data_loader,
                loss_fn,
                optimizer,
                scheduler,
                n_examples):
    """
    回歸器的主要訓練流程
    Main training process of bert sentiment regressor
    """
    model = model.train()

    losses = []
    correct_predictions = 0.
    for batch in data_loader:
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        preds = {
            aspect: output
            for aspect, output in outputs.items()
        }
        targets = {
            aspect: batch[f"{aspect}_target"].to(DEVICE)
            for aspect in preds.keys()
        }
        aspect_losses = {
            aspect: loss_fn(outputs[aspect], targets[aspect])
            for aspect in preds.keys()
        }

        loss = torch.stack([val for _, val in aspect_losses.items()]).sum()
        losses.append(loss.item())

        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return np.mean(losses)

In [11]:
def eval_model(model,
               data_loader,
               loss_fn,
               n_examples):
    """
    回歸器訓練時，每個 epoch 評估流程
    Main evaluate process in training of bert sentiment regressor
    """
    model = model.eval()

    losses = []
    correct_predictions = 0.
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            preds = {
                aspect: output
                for aspect, output in outputs.items()
            }
            targets = {
                aspect: batch[f"{aspect}_target"].to(DEVICE)
                for aspect in preds.keys()
            }
            aspect_losses = {
                aspect: loss_fn(outputs[aspect], targets[aspect])
                for aspect in preds.keys()
            }

            loss = torch.stack([val for _, val in aspect_losses.items()]).sum()
            losses.append(loss.item())

    return np.mean(losses)

In [12]:
TRAIN = pd.read_json("./data/train_set_lng.json")
TRAIN = TRAIN.sample(frac=1).reset_index(drop=True)
VAL = pd.read_json("./data/test_set_lng.json")
VAL = VAL.sample(frac=1).reset_index(drop=True)
TRAIN = TRAIN.append(VAL[500:]).reset_index(drop=True)
VAL = VAL.iloc[:500]

In [13]:
TRAIN.head(3)

,index,beer/ABV,beer/beerId,beer/brewerId,beer/name,beer/style,review/appearance,review/aroma,review/overall,review/palate,review/taste,review/text,review/timeStruct,review/timeUnix,user/ageInSeconds,user/birthdayRaw,user/birthdayUnix,user/gender,user/profileName,review_appearance,review_aroma,review_overall,review_palate,review_taste,text_length
0,7503,15.0,32222,765,Trafalgar Korruptor,American Strong Ale,3.0,3.5,2.5,3.5,2.5,Sampled at Toronto Beer fest '08.\t\tA - Clear...,"{'min': 20, 'hour': 20, 'mday': 2, 'sec': 10, ...",1220386810,NaN,None,NaN,Male,allergictomacros,1,1,1,1,1,70
1,29889,7.2,5441,1199,Founders Centennial IPA,American IPA,4.0,4.0,4.0,4.0,4.0,Pours a moderately hazy orange color with a th...,"{'min': 5, 'hour': 5, 'mday': 24, 'sec': 6, 'y...",1306213506,NaN,None,NaN,None,BrewerB,2,2,2,2,2,67
2,11802,4.7,45116,2554,Matso's Smokey Bishop,Munich Dunkel Lager,NaN,NaN,NaN,NaN,NaN,Drank from my Duvel Tulip\t\tA - Poured a dark...,"{'min': 54, 'hour': 12, 'mday': 19, 'sec': 18,...",1321707258,NaN,None,NaN,None,PolarBear3,3,3,3,3,3,95


In [14]:
MODEL = BeerRateRegressor()
MODEL.to(DEVICE)

TRAIN_DATA_LOADER = create_data_loader(TRAIN, MAX_LEN, BATCH_SIZE)
VAL_DATA_LOADER = create_data_loader(VAL, MAX_LEN, BATCH_SIZE)

OPTIMIZER = AdamW(MODEL.parameters(), lr=2e-5, correct_bias=False)# 須完成之部分
TOTAL_STEPS = len(TRAIN_DATA_LOADER) * EPOCHS# 須完成之部分
SCHEDULER = get_linear_schedule_with_warmup(
    OPTIMIZER,
    num_warmup_steps=TOTAL_STEPS // 10, #0,
    num_training_steps=TOTAL_STEPS
)
LOSS_FN = nn.MSELoss().to(DEVICE)# 須完成之部分

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
""" 訓練流程 """
# 須完成之部分
BEST_LOSS = np.inf

for epoch in range(EPOCHS):
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    print('-' * 10)

    train_loss = train_epoch(
        MODEL,
        TRAIN_DATA_LOADER,
        LOSS_FN,
        OPTIMIZER,
        SCHEDULER,
        len(TRAIN)
    )
    print(f"Train loss {train_loss}")

    val_loss = eval_model(
        MODEL,
        VAL_DATA_LOADER,
        LOSS_FN,
        len(VAL)
    )
    print(f"Val   loss {val_loss}")
    print()

    if val_loss < BEST_LOSS:
        #os.makedirs('model', exist_ok=True)
        torch.save(MODEL.state_dict(), os.path.join('data', 'best_model_state_regress.bin'))
        BEST_LOSS = val_loss

Epoch 1/5
----------
Train loss 3.6731902374786642
Val   loss 2.693889420479536

Epoch 2/5
----------
Train loss 2.5596789863314564
Val   loss 2.7576363906264305

Epoch 3/5
----------
Train loss 2.236074141883511
Val   loss 2.939439419656992

Epoch 4/5
----------
Train loss 1.9192565586694843
Val   loss 3.1021834649145603

Epoch 5/5
----------
Train loss 1.7072418408772987
Val   loss 3.1145950257778168

